In [22]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import torch
from torchsummary import summary
from thop import profile
import numpy as np
import time

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
def measure_time_cpu(model, device, rep = 10):
    model = model.to(device=device)
    x = torch.rand((2, 16, 9), device=device)
    timings=np.zeros((rep,1))
    for i in range(rep):    
        start_time = time.time()
        out = model(x)
        timings[i] = time.time() - start_time
    mean_syn = np.sum(timings) / rep
    std_syn = np.std(timings)
    return mean_syn, std_syn

def measure_time_gpu(model, device, rep):
    model = model.to(device=device)
    dummy_input = torch.randn(2, 16, 9, dtype=torch.float).to(device)
    # INIT LOGGERS
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    repetitions = rep
    timings=np.zeros((repetitions,1))
    #GPU-WARM-UP
    for _ in range(100):
        _ = model(dummy_input)
    # MEASURE PERFORMANCE
    with torch.no_grad():
        for rep in range(repetitions):
            starter.record()
            _ = model(dummy_input)
            ender.record()
            # WAIT FOR GPU SYNC
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)
            timings[rep] = curr_time
    mean_syn = np.sum(timings) / repetitions
    std_syn = np.std(timings)
    return mean_syn, std_syn

In [24]:
from model.model import Conv1dAnomalyTransformer, OnlyAnomalyTransformer

model = Conv1dAnomalyTransformer(
    d_model=10,
    layer=10,
    num_class=7,
    with_time=False,
    use_emb=True,
    add_norm=True,
    in_dim=9,
    emb_size=64,
    win_size=16
).to('cuda')

init Conv1dTransformerEncoder with 7 classes


In [25]:
measure_time_gpu(model, 'cuda', rep=1000)

(19.954039329528808, 1.7465905460901445)

In [26]:
def calculate_MAC(parameter_count, model_weights):
    mac = parameter_count * model_weights
    return mac

def calculate_parameter_count(model):
    parameter_count = sum(p.numel() for p in model.parameters())
    return parameter_count

def calculate_model_weights(model):
    model_weights = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return model_weights

parameter_count = calculate_parameter_count(model)
model_weights = calculate_model_weights(model)
mac = calculate_MAC(parameter_count, model_weights)

print(f"Parameter count: {parameter_count}")
print(f"Model weights: {model_weights}")
print(f"MAC: {mac}")

Parameter count: 160356
Model weights: 160356
MAC: 25714046736


In [31]:
from torchprofile import profile_macs
from thop import profile

macs, params = profile(model, inputs=(input_tensor,))
print(f"MACs: {f'{macs:,}' }, Parameters: {f'{params:,}' } using THOP")

inputs = torch.randn(2, 16, 9).to('cuda')
macs = profile_macs(model.to(inputs.device), inputs)
print(f"MACs: {macs} using torchprofile")

[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
MACs: 2,035,648.0, Parameters: 160.156 using THOP
MACs: 3977216 using torchprofile
